<div style="text-align: center;">
    <h1>Registre national des entreprises (RNE)</h1>
</div>


- [ ] verif pour une entreprise qui a plusieurs entreprises que le parsing est bien fait (ie que tout est pris)

## Import des librairies

In [1]:
import pandas as pd
import numpy as np
import json 
import os
from tqdm import tqdm

# 1. Téléchargement

Ce registre vient de [l'Institut National de la Propriété Industrielle](https://data.inpi.fr/) qui donne accès à de nombreuses données sur les entreprises.

Les données peuvent être consultées sur le web, sur le serveur de l'INPI ou par API. Il aurait été pratique d'utiliser l'API pour des questions de facilité de prise en main, de mise à jour des données etc ...

Néanmoins la [documentation officielle](https://www.inpi.fr/sites/default/files/documentation_technique_API_formalit%C3%A9s_v2.5.pdf) indique qu'il est impossible de filtrer géographiquement les établissements dans les méthodes GET et impose une limite de 10 Go / jour / utilisateur, ce qui est bloquant pour notre application.


On va donc utiliser les serveurs et télécharger les donnéees avec le protocole FTP.

Pour cela, il faut créer un compte INPI et faire la demande pour avoir les accès.

Une fois le compte crée, on peut accéder au serveur FTP de l'INPI (voir [ici](https://data.inpi.fr/content/editorial/Serveur_ftp_entreprises)).

Les données user_id / mdp sont retrouvables sur [ce lien](https://data.inpi.fr/espace_personnel/acces).

On télécharge le registre des Créations, modifications, cessations (CMC) d'entreprises dont l'INPI donne la description suivante :



- Couverture
    - Inscriptions relatives aux créations, modifications et cessations, déposées par les entreprises
    
- Données disponibles
    - Stock des dossiers des **personnes morales et personnes physiques actives à jour** à la date du 4 mai 2017  (stock) pour les entreprises commerciales

- Volumétrie (estimation)
    - Environ 5 millions de sociétés actives pour le stock
    - Environ 2 millions d'inscriptions par an
    
- Mise à jour quotidienne

- Format de livraison : Json


Pour le télécharger, on peut s'inspirer du **code suivant** :

```bash
# connexion au serveur ftp de l'inpi
sftp user_id@www.inpi.net

# accepter la connexion et rentrer le mot de passe

# création d'un dossier sur la machine locale
lmkdir rne

# navigation dans ce dossier 
llcd rne

# téléchargement des fichiers
get stock RNE formalité.zip 

# le téléchargement dure 15 min environ

# décompression des données
unzip stock RNE formalité.zip 
````


# 2. Nettoyage des données

Les données obtenues sont une collection de jsons, on aimerait pouvoir les retraiter pour pouvoir les utiliser plus efficacement.

Cela implique de :

- **destructurer les informations pour aller à la maille des établissements** 

    On différencie ici les entreprise (identifiées par leur SIREN) des établissements (identifiées par leur SIRET = SIREN + NIC), plus d'infos [ici](https://entreprendre.service-public.fr/vosdroits/F32135#:~:text=Siret%20signifie%20Syst%C3%A8me%20d'identification,num%C3%A9ro%20interne%20de%20classement%20Insee).

- **ne garder que les infos pertinentes, à savoir à la maille d'un établissement** :



    | Siret              | Dénomination Usuelle | Code INSEE Commune | Code APE | Adresse            |diffusionCommerciale           |
    |--------------------|----------------------|---------------------|----------|--------------------|--------------------|


- **convertir en un seul fichier plus efficace (on a choisi le format parquet)**





## Conversion des json en parquet

Pour accélerer le temps de traitement des fichiers et prendre moins de stockage, on convertit tous les json en parquet

In [33]:
# install dependencies without verbose output
! pip install -q pyarrow fastparquet


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


**NB:** On peut remplacer la valeur de la variable 
```Python
base_path 
````

Par le chemin relatif des fichiers décompressés

In [ ]:
%%time
base_path = "../../rne/rne_deflated"


# convert json to parquet
def convert_json_to_parquet(json_path, parquet_path):

    # read json
    with open(json_path, "r") as f:
        json_data = json.load(f)

    # convert to dataframe
    df = pd.DataFrame(json_data)

    # save as parquet
    df.to_parquet(parquet_path)

# convert all json files to parquet
for root, dirs, files in os.walk(base_path):
    for file in tqdm(files):
        if file.endswith(".json"):
            json_path = os.path.join(root, file)
            parquet_path = json_path.replace(".json", ".parquet")
            convert_json_to_parquet(json_path, parquet_path)




### Vérification que la transcription s'est bien faite

In [3]:
# list of all parquet files paths
parquets = [os.path.join(base_path,f) for f in os.listdir(base_path) if f.endswith(".parquet")]

# choose a random one and read it

rand = np.random.randint(0, len(parquets))

df = pd.read_parquet(parquets[rand])

df


,updatedAt,id,formality,siren
0,2023-05-15T10:07:58+02:00,6446b1073fa153b9ae00f31d,"{'content': {'exploitation': None, 'formeExerc...",950608935
1,2023-05-15T10:07:55+02:00,63aec72f4c8b4e83ab11f64b,"{'content': {'exploitation': None, 'formeExerc...",950608745
2,2023-05-15T10:07:56+02:00,63aec7302336741e761a3433,"{'content': {'exploitation': None, 'formeExerc...",950608760
3,2023-06-01T13:37:05+02:00,63aec730673de62848173aff,"{'content': {'exploitation': None, 'formeExerc...",950608927
4,2023-06-16T05:42:15+02:00,63aec7316c5cbb032e11c173,"{'content': {'exploitation': None, 'formeExerc...",950609552
...,...,...,...,...
99944,2023-04-19T18:04:24+02:00,64401108ac36a1fe2f061cff,"{'content': {'exploitation': None, 'formeExerc...",951769009
99945,2023-04-21T12:36:00+02:00,644267101481c6e92c01de6f,"{'content': {'exploitation': None, 'formeExerc...",951768142
99946,2023-04-26T15:33:40+02:00,64492834822ca95bf2065602,"{'content': {'exploitation': None, 'formeExerc...",951768597
99947,2023-05-04T16:31:12+02:00,6453c1b079ed744a040a1c33,"{'content': {'exploitation': None, 'formeExerc...",951768134


## Passage de données par entreprise à des données par établissement

Pour pouvoir avoir des filtres géographiques, on a besoin d'avoir le détail par établissement.

Or les données sont regroupées pour l'instant à l'échelle d'une entreprise. 

On cherche donc la meilleure manière de faire ce filtre, on peut se référer à la [documentation de l'inpi](https://www.inpi.fr/sites/default/files/documentation_technique_API_formalit%C3%A9s_v2.5.pdf) pour voir comment sont structurées les données.

Sur le site, de l'INPI, on peut aussi consulter leur [dictionnaire des données](https://www.inpi.fr/sites/default/files/Dictionnaire_de_donnees_2023_avril.xls).

#### Fonction pour convertir les données associées à une entreprise en json pour pouvoir parser facilement

In [25]:


class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            if obj.dtype == object:
                # Handle NumPy array with dtype=object
                return obj.tolist()
            else:
                # Handle other NumPy arrays
                return obj.tolist()
        return json.JSONEncoder.default(self, obj)

def convert_np_to_list(obj):
    if isinstance(obj, dict):
        return {key: convert_np_to_list(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_np_to_list(item) for item in obj]
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj
    


# Conversion and saving
with open("temp.json", "w") as f:

    # Choose a random row
    rand = np.random.randint(0, len(df))

    # Convert numpy arrays to lists recursively
    converted_text = convert_np_to_list(df.iloc[rand].to_dict())

    # Save output as indented JSON
    json.dump(converted_text, f, indent=4, cls=NumpyEncoder)

            

#### Fonctions pour parser

In [33]:
# pour une entrepise sans établissement secondaire, retourne les informations de l'entreprise
def parse_etablissement(etablissementPrincipal,diffusionCommerciale,nomCommercial,codeApe):

    # certaines entreprises n'ont pas de nom commercial (ex : siren 949 299 903)
    try :
        siret = etablissementPrincipal["descriptionEtablissement"]["siret"]
    except :
        return None

    # certaines entreprises n'ont pas d'adresse ex : siren 950567479
    try :
        adresse_base = etablissementPrincipal["adresse"]

        # Check for None values and handle them
        elements = [adresse_base.get(key, "") for key in ["numVoie", "typeVoie", "voie", "codePostal", "commune"]]

        # Construct the address string, skipping None values
        adresse = " ".join(elements) if None not in elements else ""
    except :
        return None

    try :
        codeInseecommune = adresse_base["codeInseeCommune"]
    except :
        return None

    return {siret: {"nomCommercial": nomCommercial, "adresse": adresse, "codeInseecommune": codeInseecommune, "codeApe": codeApe, "diffusionCommerciale": diffusionCommerciale}}

# liste des codes rolePourEntreprise d'établissement fermé
rolePourEntreprise_fermes = ["11","12","13","14","15","16"]


# pour une entreprise avec établissement secondaire, retourne les informations de chaque établissement
def parse_etablissements_secondaires(autresEtablissements, diffusionCommerciale, formality):

    # one dict to store them all
    etablissements = {}

    # pour chaque établissement
    for etablissement in autresEtablissements:
        
        # si l'établissement est fermé, on ne le prend pas en compte
        if etablissement["descriptionEtablissement"]["rolePourEntreprise"] in rolePourEntreprise_fermes:
            continue

        # etablissement ouvert
        else :

            try :
    
                adresse_base = etablissement["adresse"]

                # Check for None values and handle them
                elements = [adresse_base.get(key, "") for key in ["numVoie", "typeVoie", "voie", "codePostal", "commune"]]

                # Construct the address string, skipping None values
                adresse = " ".join(elements) if None not in elements else ""


            except :

                adresse = None

            # si l'adresse n'est pas en France, on ne la prend pas en compte
            try :
                codeInseecommune = etablissement["adresse"]["codeInseeCommune"]
            except :
                codeInseecommune = None

            siret = etablissement["descriptionEtablissement"]["siret"]

            codeApe = formality["content"]["personneMorale"]["identite"]["entreprise"]["codeApe"]
            
            nomCommercial = etablissement["descriptionEtablissement"]["nomCommercial"]

            if nomCommercial == None:
                nomCommercial = formality["content"]["personneMorale"]["identite"]["entreprise"]["denomination"]

            diffusionCommerciale = formality["diffusionCommerciale"]
        
            etablissements[siret] = {"nomCommercial": nomCommercial, "adresse": adresse, "codeInseecommune": codeInseecommune, "codeApe": codeApe, "diffusionCommerciale": diffusionCommerciale}

    return etablissements

# retourne les informations de l'entreprise concernée par la cessation
def parse_formality(formality):


    # personne morale
    if formality["content"]["personneMorale"] == None:
        return None
    

    # cas 1 : entreprise sans établissement secondaire
    if (not isinstance(formality["content"]["personneMorale"]["autresEtablissements"],np.ndarray)) and formality["content"]["personneMorale"]["etablissementPrincipal"] != None:

        codeApe = formality["content"]["personneMorale"]['identite']["entreprise"]["codeApe"]
        denomination = formality["content"]["personneMorale"]["identite"]["entreprise"]["denomination"]
        diffusionCommerciale = formality["diffusionCommerciale"]

        return parse_etablissement(formality["content"]["personneMorale"]["etablissementPrincipal"],diffusionCommerciale,denomination,codeApe)
    
    
    # cas 2 : entreprise avec établissement secondaire
    elif isinstance(formality["content"]["personneMorale"]["autresEtablissements"],np.ndarray) :


        return parse_etablissements_secondaires(formality["content"]["personneMorale"]["autresEtablissements"], formality["diffusionCommerciale"],formality)


    # cas 3 : entreprise avec mauvaise saisie : ni établissement principal, ni établissement secondaire
    # ex : siren 949759906
    else :
        return None

#### Debugging 

In [28]:
# generate 10000 random numbers
rand = np.random.randint(0, len(df), 100000)

# for each random number, parse the content and create a dataframe
df_parsed = pd.DataFrame(columns=["siret", "nomCommercial", "adresse", "codeInseecommune", "codeApe", "diffusionCommerciale","row"])

for i in tqdm(rand):

    row = df.iloc[i]

    try :
        parsed = parse_formality(row["formality"])
    except :
        print("Error on row {}".format(i))
        break

    if parsed == None:
        continue

    # concatenate the row number to the parsed data
    for key, value in parsed.items():
        value["row"] = i

    # append to the dataframe
    pd.concat([df_parsed, pd.DataFrame.from_dict(parsed, orient="index")], axis=0)


100%|██████████| 100000/100000 [00:18<00:00, 5374.02it/s]


In [29]:
error_row = 9817

# Conversion and saving
with open("temp.json", "w") as f:


    # Convert numpy arrays to lists recursively
    converted_text = convert_np_to_list(df.iloc[error_row].to_dict())

    # Save output as indented JSON
    json.dump(converted_text, f, indent=4, cls=NumpyEncoder)

parse_formality(df.iloc[error_row]["formality"])     

{'95077902500010': {'nomCommercial': 'SCI IMETAMANT',
  'adresse': '',
  'codeInseecommune': '76259',
  'codeApe': '6820B',
  'diffusionCommerciale': True}}

Fin du debugging : on enlève le temp

In [30]:
if os.path.exists('temp.json'):
    os.remove('temp.json')

#### Test sur un fichier

In [54]:
%%time

def parse_parquet(parquet_path):

    # lecture du parquet
    df = pd.read_parquet(parquet_path)

    # liste des résultats
    results = []

    # pour chaque entreprise 
    for i, row in df.iterrows():

        # parse the formality
        parsed = parse_formality(row["formality"])

        if parsed == None or parsed == {}:
            continue    

        else :
            # extend the dict with the parsed data
            results.append(parsed)


    # converts to dataframe
    df = pd.DataFrame([(key, *value.values()) for dct in results for key, value in dct.items()], columns=["siret", "nomCommercial", "adresse", "codeInseecommune", "codeApe", "diffusionCommerciale"])

    return df



# choose a random parquet file
rand = np.random.randint(0, len(parquets))

# parse it
parse_parquet(parquets[rand])



CPU times: user 4.97 s, sys: 1.4 s, total: 6.37 s
Wall time: 7.1 s


,siret,nomCommercial,adresse,codeInseecommune,codeApe,diffusionCommerciale
0,39517329700015,L'ESQUIROL DE SAINT SAUVEUR,,24499,6820B,True
1,39517431100013,SCI MSB RESIDENCE,7 RUE FELIX GERMAIN 30200 BAGNOLS-SUR-CEZE,30028,6820B,True
2,39517434500011,GIE TROUVE ALDEBERT,139 AV DE VERDUN 92130 ISSY-LES-MOULINEAUX,92040,454J,True
3,39517410500019,CEVEN'AVENTURE PASSION,,07334,926C,True
4,39517378400012,SCI MOZART,2 RUE PASTEUR 95170 DEUIL-LA-BARRE,95197,6820A,True
...,...,...,...,...,...,...
31297,39827400100013,J M J,26 RUE DE LA RONCE 77580 VOULANGIS,77529,454J,True
31298,39827300300010,TENNIS PARADISE,2 RTE D HEYRIEUX 69780 TOUSSIEU,69298,926C,True
31299,39827355700015,ERBECO,36 LOT LES AMARILYS 97354 REMIRE-MONTJOLY,97309,454D,True
31300,39827374800010,SNC TABAC NEW KHEDIVE,67 RUE D ANTIBES 06400 CANNES,06029,554A,True


#### Parsing de tous les fichiers

In [56]:
%%time

# loops over all parquet files and parse them
for parquet in tqdm(parquets):
    df = parse_parquet(parquet)
    
    # save as parquet
    df.to_parquet(parquet.replace(".parquet", "_parsed.parquet"))



100%|██████████| 241/241 [31:54<00:00,  7.94s/it]

CPU times: user 21min, sys: 7min 29s, total: 28min 29s
Wall time: 31min 54s


## Création d'un seul parquet

In [57]:
parsed_parquets = [os.path.join(base_path,f) for f in os.listdir(base_path) if f.endswith("_parsed.parquet")]

print("Nombre de parquets parsés : {}".format(len(parsed_parquets)))

Nombre de parquets parsés : 241


In [58]:
%%time 


# concat all parsed parquets into a single parquet file
df = pd.concat([pd.read_parquet(parquet) for parquet in tqdm(parsed_parquets)])

# save as parquet
df.to_parquet(os.path.join(base_path, "final_dataset_parsed.parquet"))

100%|██████████| 241/241 [00:03<00:00, 71.80it/s]


CPU times: user 6.8 s, sys: 2.14 s, total: 8.93 s
Wall time: 9.63 s


## Comparaison de la taille pris par les fichiers

In [59]:
jsons = [os.path.join(base_path,f) for f in os.listdir(base_path) if f.endswith(".json")]
parsed_parquets = [os.path.join(base_path,f) for f in os.listdir(base_path) if f.endswith("_parsed.parquet")]
parquets = [os.path.join(base_path,f) for f in os.listdir(base_path) if f.endswith(".parquet")]
original_parquets = [parquet for parquet in parquets if parquet not in parsed_parquets]

print("Poids des jsons : {} Mo".format(round(sum([os.path.getsize(json) for json in jsons]) / 1000000, 2)))

print("Poids des parquets originaux : {} Mo".format(round(sum([os.path.getsize(parquet) for parquet in original_parquets]) / 1000000, 2)))

print("Poids des parquets parsés : {} Mo".format(round(sum([os.path.getsize(parquet) for parquet in parsed_parquets]) / 1000000, 2)))

print("Poids du parquet final : {} Mo".format(round(os.path.getsize(os.path.join(base_path, "final_dataset_parsed.parquet")) / 1000000, 2)))



Poids des jsons : 108100.5 Mo
Poids des parquets originaux : 8483.95 Mo
Poids des parquets parsés : 712.89 Mo
Poids du parquet final : 346.08 Mo


In [60]:
df = pd.read_parquet(os.path.join(base_path, "final_dataset_parsed.parquet"))

df

,siret,nomCommercial,adresse,codeInseecommune,codeApe,diffusionCommerciale
0,40825446400014,SCI GRAISSAC,8 RUE DU ROCHER 75008 PARIS 8,75108,6820B,True
1,40825448000010,LE SAINTES SCARBES,10 PL SAINTES SCARBES 31000 TOULOUSE,31555,553A,True
2,40825449800020,HELIOS PEINTURE,13 RUE DE LA VICTOIRE 93150 LE BLANC-MESNIL,93007,454J,True
3,40825480300013,SCP VETERINAIRES DUBOST F ET TARDIEUX MC,,13110,7500Z,True
4,40825387000013,IC 3A,1 RUE ROBERT FOURNERON 07400 LE TEIL,07319,742C,True
...,...,...,...,...,...,...
12799,78849706300016,COBAMET,92 RUE CAROLINE FOLLET 80160 CONTY,80211,4332B,True
12800,78849721200019,MASSAGE CHINOIS 6,6 RUE DE LANCRY 75010 PARIS 10,75110,9604Z,True
12801,78849780800014,ID'S WORKS,3 CHE DE LA POUPETTERIE 86800 SEVRES-ANXAUMONT,86261,7311Z,True
12802,78849702200012,LES BEAUX PINS,14 RUE DES BRUYERES 33450 SAINT-LOUBES,33433,6820B,True


## Clean du parquet final

In [61]:
# check for duplicates

print("Nombre de lignes : {}".format(len(df)))

print("Nombre de lignes uniques : {}".format(len(df.drop_duplicates())))

Nombre de lignes : 7831781
Nombre de lignes uniques : 7831770


In [70]:
df.drop_duplicates().to_parquet("../Données nationales/RNE.parquet",index=False)

In [69]:
# check siret 45132133500023 (Carrefour)

df[df["siret"] == "32522172900016"]

,siret,nomCommercial,adresse,codeInseecommune,codeApe,diffusionCommerciale
57,32522172900016,SA SBME INTERMARCHE,37 RUE HERVE DE GUEBRIANT 29800 LANDERNEAU,29103,4711F,True


In [74]:
df_sirene = pd.read_csv("../Données nationales/localisation_et_naf_siret.csv", sep=",")



In [75]:
df_sirene

,siret,plg_code_commune,y_latitude,x_longitude,activitePrincipaleEtablissement
0,9575031100300,37050,47.333916,0.706212,NaN
1,30499363700111,38229,45.211540,5.790160,NaN
2,21440132500037,44132,47.252959,-2.314688,NaN
3,31000011200048,73008,45.695475,5.908980,NaN
4,31035364400019,73008,45.700385,5.901790,NaN
...,...,...,...,...,...
17014677,48919692300010,77238,48.927893,3.139255,02.10Z
17014678,48919693100013,95680,49.009962,2.398370,NaN
17014679,48919694900015,69248,46.032473,4.310816,NaN
17014680,48919694900023,69248,46.035427,4.296627,NaN


In [77]:
df_sirene[df_sirene["siret"] == "32522172900016"]

,siret,plg_code_commune,y_latitude,x_longitude,activitePrincipaleEtablissement
